#### This is a framework for function approximation. Function is y=sin(x)
Regularization: L2 didn't make much of a difference and Dropout did not work

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import sys
import matplotlib.pyplot as plt

L1_SIZE       = 20           # How many nodes in layer 1
BATCH_SIZE    = 100
EPOCHS        = 1000
TEST_PCT      = 0.20
VAL_PCT       = 0.10
FEATURE_COUNT = 1
LEARNING_RATE = .01
SUMMARIES_DIR = '/home/tom/tf_logs' # where to store Tensorboard data
LAMBDA        = 0.              # Regularization parameter

#### Generate the data

In [ ]:
# Create training data: y = sin(x)
x_ = np.random.random((1000,1))*10
y_ = np.sin(x_)

data = np.concatenate([x_,y_] ,axis=1)

In [ ]:
# Split into train and test
train, test = train_test_split(data,  test_size=TEST_PCT)
train, val  = train_test_split(train, test_size=VAL_PCT)

train_x = train[:,0].reshape(-1,1)
train_y = train[:,1].reshape(-1,1)
val_x   = val[:,0].reshape(-1,1)
val_y   = val[:,1].reshape(-1,1)
test_x  = test[:,0].reshape(-1,1)
test_y  = test[:,1].reshape(-1,1)

In [ ]:
# Training data should look like a sine wave
a = plt.scatter(train_x, train_y, s=1, color='blue')
plt.title('Training data: Y=sin(X)')
plt.show()

In [ ]:
# Set up the network
tf.reset_default_graph()
x  = tf.placeholder("float", shape=[None, FEATURE_COUNT])
y_ = tf.placeholder("float", shape=[None, 1])

l1_w   = tf.Variable(tf.truncated_normal([FEATURE_COUNT, L1_SIZE], dtype=tf.float32))
l1_b   = tf.Variable(tf.zeros([1,L1_SIZE], dtype=tf.float32))
l1_act = tf.nn.tanh(tf.matmul(x,l1_w) + l1_b)
#l1_act = tf.nn.dropout(l1_act, keep_prob=0.6)

l2_w = tf.Variable(tf.truncated_normal([L1_SIZE,1], dtype=tf.float32))
l2_b = tf.Variable(tf.zeros([1,1]))
l2_out = tf.matmul(l1_act, l2_w) + l2_b

#cost = tf.div(tf.add(tf.reduce_sum(tf.square(l2_out - y_)),
#                     tf.mul(LAMBDA, tf.reduce_sum(tf.square(l1_w)))),
#              2*train.size)

cost = tf.reduce_mean(tf.square(l2_out - y_))
tf.scalar_summary('cost', cost)

# Optimizer
optimize = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE).minimize(cost)

merged = tf.merge_all_summaries()

In [ ]:
count = 1
num_training_batches = int(len(train_x) / BATCH_SIZE)
print('{} epochs of {} iterations with batch size {}'.format(EPOCHS,num_training_batches,BATCH_SIZE))

with tf.Session() as sess:
    train_writer = tf.train.SummaryWriter(SUMMARIES_DIR + '/train', sess.graph)
    sess.run(tf.initialize_all_variables())
    for i in range(EPOCHS):
        a,b = shuffle(train_x,train_y)
        for j in range(num_training_batches):
            x_mini = a[j*BATCH_SIZE:j*BATCH_SIZE+BATCH_SIZE]
            y_mini = b[j*BATCH_SIZE:j*BATCH_SIZE+BATCH_SIZE]
            sess.run(optimize, feed_dict = {x: x_mini, y_: y_mini})
        if i % 10 == 0:
            s, m = sess.run([cost, merged], feed_dict = {x: val_x, y_: val_y})
            train_writer.add_summary(m, count)
            count += 1
    s,y_pred = sess.run([cost, l2_out], feed_dict = {x: test_x, y_: test_y})
    print('Final score: {:.1f}'.format(s))
train_writer.close()
!aplay /usr/share/sounds/bicycle_bell.wav

In [ ]:
# See what the data looks like
a = plt.scatter(test_x, y_pred, s=1, color='blue',label='Predicted')
b = plt.scatter(test_x, test_y, s=1, color='red',label='Actual')
plt.legend(loc='lower right')
plt.show()